# Packages

In [1]:
import pandas as pd
import numpy as np
import pickle
import networkx as nx
import time
import itertools
import random
import copy
import sys
import os
import csv
from scipy.spatial import distance
import scipy.sparse as sp
from scipy.spatial import distance
import scipy.sparse as sp
import gurobipy as gp
from gurobipy import GRB
from st_network2 import Many2Many
from python_tsp.exact import solve_tsp_dynamic_programming
from python_tsp.distances import great_circle_distance_matrix
from trip_prediction import trip_prediction

# import osmnx as ox
# from joblib import Parallel, delayed, load, dump
# from mpl_toolkits.basemap import Basemap

# Information
<code>tau</code>:= zone-to-zone travel time numpy array<br>
<code>tau2</code>:= zone-to-zone travel time numpy array with diagonal elements equal to one<br>
<code>ctr</code>:= a dictionary with key as zones and values be the set of neighbor zones for the key including the key itself<br>
<code>fixed_route_D</code>:= Dictionary with key as fixed route bus driver ID, with with value as $(t, s)$ pair where $t$ is the time and $s$ is the node for each stop of scheduled bus route
### <code>config</code> data structure:
<code>beta</code>:= time flexibility budget<br>
<code>S</code>:= Number of zones<br>
<code>T</code>:= Number of time steps<br>
<code>VEH_CAP</code>:= Vehicle capacity<br>
<code>FIXED_ROUTE</code>:= True if use fixed bus route configuration<br>
<code>REPEATED_TOUR</code>:= True if use repeated bus route configuration<br> 
<code>TIME_LIMIT</code>:= Time limit of solver<br> 
<code>MIP_GAP</code>:= The optimality gap

### <code>Rider</code> and <code>Driver</code> Numpy array, every row has these columns:
0: user index<br>
1: IGNORE!<br>
2: earliest departure time<br>
3: latest arrival time<br>
4: origin<br>
5: destination<br>
6 (Rider only): tranfer limit

If <code>REPEATED_TOUR</code>: earliest departure time and latest arrival time should be only for the first single tour for the each driver, not for the entire time horizon.

In [27]:
config = dict()
config["S"] = 69
config["T"] = 185
config["beta"] = 10  # x DELTA_t (mins) time flexibility budget
config["VEH_CAP"] = 10
config["FIXED_ROUTE"] = False
config["REPEATED_TOUR"] = False
config["TIME_LIMIT"] = 500
config["MIP_GAP"] = 0.2
DELTA_t = 1  # discrete time interval in minutes

### Load neighbor nodes information

In [28]:
ctr_info = pickle.load(open(r"Data\temp\Station.p", "rb"))
ctr = dict()
zone_id = list(ctr_info.keys())
for i in range(len(ctr_info)):
    ctr_info[i]["neighbours"].append(zone_id[i])
    ctr[i] = list(e for e in ctr_info[i]["neighbours"] if e < config["S"])
ctr[0]

[5, 29, 40, 41, 42, 43, 53, 65, 0]

In [46]:
S = config["S"]
# graph with edge cost as shortest travel time
G_t = pickle.load(open(r"Data\temp\G_t.p", "rb"))
# graph with edge cost as shortest travel distance
G_d = pickle.load(open(r"Data\temp\G_d.p", "rb"))

tau = np.zeros((S, S))
tau2 = np.zeros((S, S))
# TO DO: convert travel time unit to the unit of many-2-many problem
for _, _, d in G_t.edges(data=True):
    d["weight"] = np.rint(d["weight"])


for i in range(S):
    for j in range(S):
        if i == j:
            tau[i, j] = 0
            tau2[i, j] = 1
        else:
            tau[i, j] = nx.shortest_path_length(
                G_t, source=i, target=j, weight="weight"
            )
            tau2[i, j] = tau[i, j]

In [48]:
# np.savetxt(r"Data\\tau_int.csv", tau, delimiter=",")
min(tau[i, j] for i in range(S) for j in range(S) if i != j)

1.0

### Mode choice model trip prediction

In [30]:
# load mode choice input data
mc_fileloc = r"mc_input_data\\"
per_dist = pd.read_csv(mc_fileloc + "dists_dat.csv", sep=",")
per_emp = pd.read_csv(mc_fileloc + "dests_emp.csv", sep=",")
mdot_dat = pd.read_csv(mc_fileloc + "mdot_trips_dc.csv", sep=",")
dests_geo = pd.read_csv(mc_fileloc + "dests_geoid.csv", sep=",")
D = pd.read_csv(mc_fileloc + "distance.csv", sep=",", index_col=[0])
# load ID converter
id_converter = pickle.load(open(r"Data\id_converter.p", "rb"))

In [31]:
(
    trips_dict_pk,
    trips_dict_op,
    transit_trips_dict_pk,
    transit_trips_dict_op,
) = trip_prediction(id_converter, per_dist, per_emp, mdot_dat, dests_geo, D)

## Generate rider information

### Rounding trip number with threshold 0.5

In [36]:
trip_dict = {
    k: int(np.round(v))
    for k, v in transit_trips_dict_pk.items()
    if int(np.round(v)) > 0
    if k[0] != k[1]
}

Rider = pd.DataFrame(columns=["ID", "NAN", "ED", "LA", "O", "D", "SL"])

N_r = sum(trip_dict.values())

O_r = list()
D_r = list()
for k, v in trip_dict.items():
    O_r += [k[0] for _ in range(v)]
    D_r += [k[1] for _ in range(v)]

Rider["ID"] = np.arange(N_r)
Rider["O"], Rider["D"] = O_r, D_r
# Rider["ED"] = 0
Rider["ED"] = np.random.randint(0, config["T"] // 5, N_r)
# Rider["LA"] = [config["T"] - 1] * N_r
Rider["LA"] = np.random.randint(config["T"] // 5 * 3, config["T"] // 5 * 4, N_r)
Rider["SL"] = [10] * N_r

Rider = Rider.fillna(999)
Rider.to_csv(r"many2many_data\Rider.csv", index=False)
Rider = Rider.to_numpy(dtype=int, na_value=999)

## Generate driver information

### Load fixed routed infomation

In [37]:
if config["FIXED_ROUTE"]:
    df_blue_1 = pd.read_csv(r"mc_input_data\blue_fr_1.csv")
    df_blue_2 = pd.read_csv(r"mc_input_data\blue_fr_2.csv")
    df_red_1 = pd.read_csv(r"mc_input_data\red_fr_1.csv")
    df_yellow_1 = pd.read_csv(r"mc_input_data\yellow_fr_1.csv")

    blue_1 = df_blue_1.to_numpy()
    blue_2 = df_blue_2.to_numpy()
    red_1 = df_red_1.to_numpy()
    yellow_1 = df_yellow_1.to_numpy()
    FR = {
        0: blue_1,
        1: blue_2,
        2: red_1,
        3: yellow_1,
    }
else:
    FR = None

In [41]:
if config["REPEATED_TOUR"]:
    Driver = pd.read_csv(r"many2many_data\Driver_rt.csv")
else:
    Driver = pd.read_csv(r"many2many_data\Driver.csv")

Driver = Driver.to_numpy(dtype=int, na_value=999)

In [ ]:
(X, U, Y, Route_D, mr, OBJ, R_match) = Many2Many(
    Rider, Driver, tau, tau2, ctr, config, fixed_route_D=FR
)

## Store Results

In [ ]:
result_loc = r"test_data\result\3_22_2021\\"
if not os.path.exists(result_loc):
    os.makedirs(result_loc)

In [ ]:
Y = [
    (
        r,
        d,
        n1 // config["S"],
        n2 // config["S"],
        n1 % config["S"],
        n2 % config["S"],
        n1,
        n2,
    )
    for (r, d, n1, n2) in Y
]
Y = sorted(Y, key=lambda x: x[2])

U = sorted(list(U), key=lambda x: x[1])

In [ ]:
for d in Route_D.keys():

    route_filename = result_loc + r"route_{}.csv".format(d)
    with open(route_filename, "w+", newline="", encoding="utf-8") as csvfile:
        csv_out = csv.writer(csvfile)
        csv_out.writerow(["t1", "t2", "s1", "s2", "n1", "n2"])
        for row in Route_D[d]:
            csv_out.writerow(row)

In [ ]:
RDL_filename = result_loc + r"Y_rdl.csv"
with open(RDL_filename, "w+", newline="", encoding="utf-8") as csvfile:
    csv_out = csv.writer(csvfile)
    csv_out.writerow(["r", "d", "t1", "t2", "s1", "s2", "n1", "n2"])
    for row in Y:
        csv_out.writerow(row)
    csv_out.writerow(
        ["Matching Rate: {}".format(mr),]
    )

In [ ]:
RD_filename = result_loc + r"U_rd.csv"
with open(RD_filename, "w+", newline="", encoding="utf-8") as csvfile:
    csv_out = csv.writer(csvfile)
    csv_out.writerow(["r", "d"])
    for row in U:
        csv_out.writerow(row)
    csv_out.writerow(
        ["Matching Rate: {}".format(mr),]
    )

# Solve the Open TSP Problem
After solving the aggregated many-to-many problem, solve the disaggregated sub-problem as an open TSP problem.

In [ ]:
sources = np.array([],)  # centroid coordinates of zones
distance_matrix = great_circle_distance_matrix(sources)
# set all elements of the first column of the distance matrix to zero
distance_matrix[:, 0] = 0
permutation, distance = solve_tsp_dynamic_programming(distance_matrix)

# Post Processing

<code>(X, U, Y, Route_D, mr, OBJ, R_match) = Many2Many(Rider, Driver, tau, tau2, ctr, config, fixed_route_D=FR)</code><br>
<code>X</code>: 1 if driver $d$ goes to link $(t_1, t_2, s_1, s_2)$, and 0 otherwise<br>
<code>U</code>: 1 if rider $r$ matches with driver $d$, and 0 otherwise<br>
<code>Y</code>: 1 if rider $r$ rides with driver $d$ on link $(t_1, t_2, s_1, s_2)$, and 0 otherwise<br>
<code>Route_D</code>: dict for each driver $d$, route link set $(t_1, t_2, s_1, s_2, n_1, n_2)$<br>
<code>mr</code>: matching rate<br>
<code>OBJ</code>: objective value<br>
<code>R_match</code>: matched riders

In [ ]:
S = config["S"]
T = config["T"]
Tmin = np.min(np.r_[Rider[:, 2], Driver[:, 2]])
Tmax = np.max(np.r_[Rider[:, 3], Driver[:, 3]])

# graph with edge cost as shortest travel distance
G_d = pickle.load(open(r"test_data\G_d.p", "rb"))

# Time expanded network
TN_t = nx.DiGraph()  # network with travel time info
TN_d = nx.DiGraph()  # network with travel distance info

# add route links into the network
for d in Route_D:
    for (_, _, _, _, n1, n2) in Route_D[d]:
        # set weight as travel time
        TN_t.add_edge(n1, n2, weight=n2 // S - n1 // S)
        TN_d.add_edge(
            n1,
            n2,
            weight=nx.shortest_path_length(
                G_d, source=n1 // S, target=n2 // S, weight="weight"
            ),
        )

In [9]:
S = 10
T = 30

In [10]:
# add waiting links into the network
for t in range(T):
    for s in range(S):
        # set weight as waiting time
        TN_t.add_edge(t * S + s, (t + 1) * S + s, weight=1.0)
        TN_d.add_edge(t * S + s, (t + 1) * S + s, weight=0.0)

od_travel_time = dict()
od_travel_dist = dict()
for i in range(S):
    od_travel_time[i] = dict()
    od_travel_dist[i] = dict()
    for j in range(S):
        od_travel_time[i][j] = -999
        od_travel_dist[i][j] = -999

In [12]:
n_lst = list(TN_t.nodes)
sp_t = {k: v for (k, v) in nx.shortest_path_length(TN_t, weight="weight")}
sp_d = {k: v for (k, v) in nx.shortest_path_length(TN_d, weight="weight")}

# update travel time influnced by bus routes
for o in sp_t:
    for d in sp_t[o]:
        if od_travel_time[o % S][d % S] == -999:
            od_travel_time[o % S][d % S] = sp_t[o][d]
        elif od_travel_time[o % S][d % S] > sp_t[o][d]:
            od_travel_time[o % S][d % S] = sp_t[o][d]

# update travel distance influnced by bus routes
for o in sp_d:
    for d in sp_d[o]:
        if od_travel_dist[o % S][d % S] == -999:
            od_travel_dist[o % S][d % S] = sp_d[o][d]
        elif od_travel_dist[o % S][d % S] > sp_d[o][d]:
            od_travel_dist[o % S][d % S] = sp_d[o][d]